In [1]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [105]:
import re
import time
import json
import base64
from redis import Redis

from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from redisvl.schema import IndexSchema
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import AutoMergingRetriever, QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import PromptTemplate

from llama_index.core.tools import QueryEngineTool, BaseTool, FunctionTool, ToolMetadata
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent

from llama_index.core.response.notebook_utils import display_source_node

In [92]:
os.environ["GROQ_API_KEY"] = os.environ.get("GROQ_API_KEY")

In [4]:
""" Convert Images to a base64 string
"""
def image_to_base64(image_path) :
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [17]:
# Initializing llama for inference as the base llm
groq_text_llm = Groq(model="llama3-70b-8192", api_key=os.environ.get("GROQ_API_KEY"))

In [16]:
# Setting groq globally in llamaindex as thr primary llm
Settings.llm = groq_text_llm

In [15]:
res = groq_text_llm.complete("What is the capital of France?")
res.text

'The capital of France is Paris.'

In [18]:
"""
Setting up the groq llm for inference on images
"""

from groq import Groq

groq_client = Groq()

IMG_DESC_PROMPT = (
    'Please describe the image as if you are explaining it to a blind child.'
    '- Use simple and friendly words.'
    '- Describe the scene in detail, including colors, shapes, people, objects, animals, and background.'  
    '- Share the emotions or feelings in the scene, if any.'
    '- Make the description vivid so they can "see" the image in their mind.'
    '- Do not use technical terms or complicated words.'
    '- Be descriptive but keep it concise, as if telling a story.'
    """- **Only provide the description of the image itself. Do not add introductory phrases like "Here's a description..." or closing phrases like "Would you like me to explain more?".**"""
)

def get_multimodal_response(
        client,
        image_str,
        text = IMG_DESC_PROMPT
    ) :
    completion = client.chat.completions.create(
        model="llama-3.2-11b-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": text
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_str}"
                        }
                    }
                ]
            }
        ],
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )
    
    return completion.choices[0].message

In [21]:
# Sample inference on an image
img_str = image_to_base64("./figures/figure-9-28.jpg")
res = get_multimodal_response(
    groq_client,
    img_str
)
print(res.content)

The image depicts a pencil sketch of a group of people standing behind two desks, with a projector screen behind them. The sketch is done in a cartoon style and features black lines and shading.

In the foreground, there are two white desks with papers on them. Behind the desks, a group of seven people stand together, all wearing light-colored jackets. They appear to be celebrating something, as they are smiling and embracing each other. 

Behind the group, a projector screen hangs on the wall, although it does not seem to be in use. The background of the image is a solid white, providing a clean and neutral backdrop for the scene.

Overall, the image suggests a joyful and celebratory atmosphere, with the group of people gathered together to mark a special occasion. 

We can tell that the people are happy because they are smiling and hugging each other. The fact that they are standing in front of the projector screen also suggests that they may be watching a presentation or slideshow t

## Retrieval for RAG

### Initializing embeddign model and redis

In [25]:
"""
Initializing the custom embedding model
"""

if not os.path.isdir("./gist_onnx") :
    OptimumEmbedding.create_and_save_optimum_model(
        "avsolatorio/GIST-Embedding-v0", "./gist_onnx", export_kwargs={'trust_remote_code': True}
    )
embed_model = OptimumEmbedding(
    folder_name="./gist_onnx"
)

In [28]:
custom_schema = IndexSchema.from_dict(
    {
        "index": {"name": "school_data_auto_bm25", "prefix": "school_doc"},
        # customize fields that are indexed
        "fields": [
            # required fields for llamaindex
            {"type": "tag", "name": "id"},
            {"type": "tag", "name": "doc_id"},
            {"type": "text", "name": "text"},
            # custom vector field for bge-small-en-v1.5 embeddings
            {
                "type": "vector",
                "name": "vector",
                "attrs": {
                    "dims": 768,
                    "algorithm": "hnsw",
                    "distance_metric": "cosine",
                },
            },
        ],
    }
)

In [31]:
redis_client = Redis.from_url("redis://localhost:6379")

In [35]:
vector_store = RedisVectorStore(
    schema=custom_schema,
    redis_client=redis_client,
)

19:42:31 redisvl.index.index INFO   Index already exists, not overwriting.


## Setting up the retrievers

In [38]:
auto_docstore = RedisDocumentStore.from_host_and_port(
    "localhost", 6379, namespace="document_store_auto"
)
bm25_docstore = RedisDocumentStore.from_host_and_port(
    "localhost", 6379, namespace="document_store_bm25"
)

In [40]:
# This is for Read mode and Quiz mode
wfw_docstore = RedisDocumentStore.from_host_and_port(
    "localhost", 6379, namespace="all_nodes"
)

In [41]:
print(len(auto_docstore.docs.items()))
print(len(bm25_docstore.docs.items()))
print(len(wfw_docstore.docs.items()))

5660
1264
1621


### Setting up storage context and index for auto merging retriever

In [44]:
auto_storage_context = StorageContext.from_defaults(docstore=auto_docstore, vector_store=vector_store)

In [47]:
auto_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

## Setting up retriever for QNA

In [50]:
bm25_retriever = BM25Retriever.from_defaults(
    docstore=bm25_docstore,
    similarity_top_k=5,
)

In [53]:
base_retriever = auto_index.as_retriever(similarity_top_k=5)
auto_retriever = AutoMergingRetriever(base_retriever, auto_storage_context, verbose=True)

In [58]:
""" Tesing auto merging retriever
"""
query_str = "What is respiration?"
retrieved_nodes = auto_retriever.retrieve(query_str)

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 1ea14154-a3e1-44f1-a7c5-5d6bfdd78e0b<br>**Similarity:** 0.8179242014879999<br>**Text:** Breathing is a part of the process of respiration during which an organism takes in oxygen-rich air and gives out air rich in carbon dioxide. The respiratory organs for the exchange of gases vary in different organisms.

During inhalation, our lungs expand and then come back to their original state as the air moves out during exhalation. Increased physical activity enhances the rate of breathing.<br>

**Node ID:** c5b385aa-d51c-4fa5-8d14-798eb58901d2<br>**Similarity:** 0.817614495754<br>**Text:** In this chapter, you learned that respiration is a vital biological process.

Keywords: Aerobic respiration, Diaphragm, Inhalation, Anaerobic respiration, Exhalation, Spiracles, Breathing rate, Gills, Tracheae, Cellular respiration, Lungs, Ribs

What you have learned: Respiration is essential for the survival of living organisms.<br>

**Node ID:** a9bcd1aa-c626-4011-953a-ffa8011c2c18<br>**Similarity:** 0.795041799545<br>**Text:** He wondered why running makes a person breathe faster. The answer to Boojho's question lies in understanding why we breathe. Breathing is a part of respiration. Let us learn about respiration.

6.1 Why Do We Respire?

In Chapter 2, you learnt that all organisms are made of small microscopic units called cells.<br>

**Node ID:** 9ab9a3ca-f7fc-42ab-b79d-ab06d9eb0d4a<br>**Similarity:** 0.790514349937<br>**Text:** It releases energy from the food. The oxygen we inhale is used to break down glucose into carbon dioxide and water. Energy is released in the process. The breakdown of glucose occurs in the cells of an organism (cellular respiration). If the food is broken down with the use of oxygen, it is called aerobic respiration.<br>

**Node ID:** 901bf311-47b6-49cd-af15-d6f802d52127<br>**Similarity:** 0.785313129425<br>**Text:** Thus, more oxygen can be supplied to the body cells, resulting in the release of more energy.

RESPIRATION IN ORGANISMS 63

Fig. 6.6 Measuring chest size
Fig. 6.7 Model to show mechanism of breathing<br>

In [57]:
""" Tesing bm25 retriever
"""
query_str = "What is pollination?"
retrieved_nodes = bm25_retriever.retrieve(query_str)

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 0eca3590-558b-4e5c-a59a-c1c5daad4bfb<br>**Similarity:** 4.269379138946533<br>**Text:** Is it to attract insects? Water. Insects visit flowers and carry away pollen on their bodies. Some of the pollen lands on the stigma of a flower of the same kind. The transfer of pollen from the anther to the stigma of a flower (a) is called pollination. If the pollen lands (b) on the stigma of the same flower or another flower of the same plant, it is called self-pollination. When the pollen of a flower lands on the stigma of a flower of a different plant of the same kind, it is called cross-pollination [Fig.<br>

**Node ID:** 750ef717-0455-4835-bae9-57e808ba550f<br>**Similarity:** 2.7358949184417725<br>**Text:** 8.7 Reproduction through spore formation in fungus

REPRODUCTION IN PLANTS 85


Activity 8.4: Male and female unisexual flowers may be present in the same plant or in different plants. Take a mustard/China rose/petunia flower and separate its reproductive parts. Study the various parts of a stamen and pistil. Anther contains pollen grains which produce male gametes. A pistil consists of stigma, style, and ovary. Ovary contains one or more ovules. The female gamete or the egg is formed in an ovule [Fig. 8.9 (b)]. In sexual reproduction, a male and a female gamete fuse to form a zygote. Boojho wants to know how the male gamete in the pollen grain reaches the female gamete present in the ovule. Filament Pollination Generally, pollen grains have a tough protective coat which prevents them from drying up. Since pollen grains are light, they can be carried by wind or insects. Flowers which contain either only pistil or only stamens are called unisexual flowers. Flowers which contain both stamens and pistil are called bisexual flowers. Corn, papaya, and cucumber produce unisexual flowers, whereas mustard, rose, and petunia have bisexual flowers. Both male gamete in the pollen grain reaches the female gamete present in the ovule. Fig. 8.9 Reproductive parts
Fig. 8.10 Pollination in flower


Boojho wants to know why flowers are generally so colourful and fragrant.<br>

**Node ID:** dfabf53c-772f-4bff-9775-b7ae92ce4ba7<br>**Similarity:** 1.463865876197815<br>**Text:** l Where is the fox? l How did it happen? l What is the fox thinking? l Who is the visitor? l What does she want to know? l What is the fox's reply? l What happens next? l Where is the goat? l Where is the fox now? l What is the goat thinking?<br>

**Node ID:** 5d22bbb3-9d98-43e3-a527-3a50e3a46841<br>**Similarity:** 1.254267692565918<br>**Text:** (a) What is the minimum interior angle possible for a regular polygon? Why? (b) What is the maximum exterior angle possible for a regular polygon?<br>

**Node ID:** 71a9735d-268f-4153-b48c-d92b36d7ed08<br>**Similarity:** 1.1815235614776611<br>**Text:** 4. “I could feel his anguish.” What could be the anguish? 5. What endeared the scientist to the writer so that he said he was looking at one of the most beautiful men in the world? 6. Read aloud the description of ‘the beautiful’ man. Which is the most beautiful sentence in the description?<br>

In [59]:
fusion_retriever = QueryFusionRetriever(
    [auto_retriever, bm25_retriever],
    similarity_top_k=3,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
)

In [60]:
""" Testing fusion retriever
"""
query_str = "What is the definition of pollination?"
retrieved_nodes = fusion_retriever.retrieve(query_str)

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

Generated queries:
Here are three search queries related to the input query:
What is the process of pollination in plants?
How do bees and other insects contribute to pollination?
What are the different types of pollination, including self-pollination and cross-pollination?


**Node ID:** 0eca3590-558b-4e5c-a59a-c1c5daad4bfb<br>**Similarity:** 0.05<br>**Text:** Is it to attract insects? Water. Insects visit flowers and carry away pollen on their bodies. Some of the pollen lands on the stigma of a flower of the same kind. The transfer of pollen from the anther to the stigma of a flower (a) is called pollination. If the pollen lands (b) on the stigma of the same flower or another flower of the same plant, it is called self-pollination. When the pollen of a flower lands on the stigma of a flower of a different plant of the same kind, it is called cross-pollination [Fig.<br>

**Node ID:** e48be09a-df7a-4256-a3c0-f23ac120c607<br>**Similarity:** 0.04946236559139785<br>**Text:** Some of the pollen lands on the stigma of a flower of the same kind. The transfer of pollen from the anther to the stigma of a flower (a) is called pollination. If the pollen lands (b) on the stigma of the same flower or another flower of the same plant, it is called self-pollination.<br>

**Node ID:** 98c8b081-851a-45c5-becb-cddde066e986<br>**Similarity:** 0.04839549075403121<br>**Text:** Fig. 8.9 Reproductive parts
Fig. 8.10 Pollination in flower<br>

## Setting up retrievers for Word for word retrieval

In [63]:
all_node_ids = list(wfw_docstore.docs.keys())
len(all_node_ids)

1621

In [61]:
def get_nodes_by_meta(filter_metadata) :
    filtered_nodes = []
    for node_id in all_node_ids:
        node = wfw_docstore.get_node(node_id)
        filter_conditions = [node.metadata.get(key) == value for key, value in filter_metadata.items()]
        if all(filter_conditions) :
            filtered_nodes.append(node)
    
    # Sort the nodes by page number
    filtered_nodes = sorted(filtered_nodes, key=lambda node: (node.metadata["chapter"], node.metadata["page_number"]))
            
    return filtered_nodes

In [65]:
filtered_nodes = get_nodes_by_meta(
    {
        "subject": "English",
        "chapter": "Chapter1",
        "doc_type": "Text"
    }
)
filtered_nodes
# # Sort nodes in page number order
# doc_nodes = sorted(filtered_nodes, key=lambda node: (node.metadata["chapter"], node.metadata["page_number"]))

[TextNode(id_='fb07daa3-df1c-4c53-9c19-82113a6a9c60', embedding=None, metadata={'chapter': 'Chapter1', 'subject': 'English', 'grade': '8th Standard', 'post_title': 'Chapter 1', 'post_description': 'The notes for chapter 1 have been uploaded', 'course_info': '8th Standard English', 'file_path': '/home/jovanzac/Jovan/OfflineProjects/Python/Machine_learning/RAG/Iris/ipynb_files/8_grade_texts/English/hehd101.pdf', 'file_name': 'hehd101.pdf', 'file_type': 'application/pdf', 'file_size': 2528583, 'creation_date': '2024-10-31', 'last_modified_date': '2024-10-31', 'page_number': 1, 'doc_type': 'Text'}, excluded_embed_metadata_keys=['image_path', 'doc_type', 'file_path', 'file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'page_number', 'file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['image_path', 'doc_type', 'file_path', 'file_name', 'file_type', 'file_size', 'creation_date', 'last_modifi

In [66]:
filtered_nodes[0].metadata

{'chapter': 'Chapter1',
 'subject': 'English',
 'grade': '8th Standard',
 'post_title': 'Chapter 1',
 'post_description': 'The notes for chapter 1 have been uploaded',
 'course_info': '8th Standard English',
 'file_path': '/home/jovanzac/Jovan/OfflineProjects/Python/Machine_learning/RAG/Iris/ipynb_files/8_grade_texts/English/hehd101.pdf',
 'file_name': 'hehd101.pdf',
 'file_type': 'application/pdf',
 'file_size': 2528583,
 'creation_date': '2024-10-31',
 'last_modified_date': '2024-10-31',
 'page_number': 1,
 'doc_type': 'Text'}

In [67]:
for doc in filtered_nodes :
    print(doc.metadata["page_number"])
    print(doc.text, end="\n"+"*"*100+"\n")

1
Before you read, there are some dates or periods of time in the history of the world that are so significant that everyone knows and remembers them. The story you will read mentions one such date and event: a war between the British and the Germans in 1914. Can you guess which war it was? Do you know which events the dates below refer to? (a) 4 July 1776, (b) 17 December 1903, (c) 6 August 1945, (d) 30 January 1948, (e) 12 April 1961, (f) 20 July 1969. The answers are on page 23.

I spotted it from a a junk shop in Bridport, a roll-top desk. The man said it was early nineteenth century, and oak. It was going for a high price, but I thought it was selling for very little money because it was in a bad condition, the roll-top in several pieces, one leg clumsily mended, scorch marks all down one side. I thought I could restore it. It would be a risk, a challenge, but I had to have it. I paid the man and brought it back to my workroom at the back of the garage. I began work on it on Chris

## Query Engine for QnA

In [70]:
query_engine = RetrieverQueryEngine.from_args(fusion_retriever, llm=Settings.llm)

In [71]:
qna_prompt_str = """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query. 
Make the response as descriptive as possible, including all relevant contexts provided.
Do not format the response in any way.
Query: {query_str}
Answer:
"""

In [74]:
new_qa_prompt = PromptTemplate(
    qna_prompt_str
)
QA_PROMPT_KEY = "response_synthesizer:text_qa_template"
# Overriding default prompt template for the query engine
query_engine.update_prompts(
    {QA_PROMPT_KEY: new_qa_prompt}
)

In [75]:
query_str = "What is the definition of pollination?"

response = query_engine.query(query_str)
response.response

Generated queries:
Here are three search queries related to the input query:
What is the process of pollination in plants?
How do bees and other insects contribute to pollination?
What are the different types of pollination, including self-pollination and cross-pollination?


'The definition of pollination is the transfer of pollen from the anther to the stigma of a flower. This process occurs when insects visit flowers and carry away pollen on their bodies, and some of this pollen lands on the stigma of a flower of the same kind.'

## Defining the ReAct agent (The agent that decides and uses the necessary tools to carry out the task requested by the user)

### Initializing the tool functions

#### a) For QnA

In [80]:
"""
Used for QnA (Mode 1 of Iris)
"""

def rag_agent(user_query) :
    """Function to respond to user's questions and academic queries"""
    print(f"Responding to te user query: {user_query}")
    
    response = query_engine.query(user_query)
    return response.response.strip()
    
#     return "The query has been successfully answered"

rag_tool = FunctionTool.from_defaults(
    fn=rag_agent,
    description= (
        "Used to respond to the user's questions and academic queries"
        "Must be used to respond to any general question, query or doubt the user has on any academic subject or topic"
        "Used for QnA type question answering."
        "The input to the tool should be in a JSON format representing the kwargs (e.g. {{'user_query': 'What is photosynthesis?'}})"
    )
)

In [81]:
rag_agent("What is photosynthesis?")

Responding to te user query: What is photosynthesis?
Generated queries:
Here are three search queries related to the input query "What is photosynthesis?":
How does photosynthesis occur in plants?
What are the importance and benefits of photosynthesis in nature?
What are the reactants and products of the photosynthesis process?


'Photosynthesis is the process by which plants prepare their own food from simple substances like water, carbon dioxide, and minerals. It is a unique process that occurs in the presence of sunlight, and it is essential for the survival of almost all living organisms. During photosynthesis, plants use energy from sunlight to convert carbon dioxide and water into carbohydrates, which are complex chemical substances that serve as food for the plant. Oxygen is also produced as a byproduct of photosynthesis, which is released into the air and is essential for the survival of other living organisms.\n\nIn more detail, photosynthesis takes place in the leaves of plants, which are the food factories of plants. The leaves contain a green pigment called chlorophyll, which helps to capture the energy of sunlight. The energy from sunlight is used to convert carbon dioxide and water into carbohydrates, which are then stored in the plant as food. This process is represented by the equation: Water an

#### b) For Word for Word retrieval

In [82]:
def number_to_ordinal(n):
    if 10 <= n % 100 <= 20:  # Handle special case for teens
        suffix = "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
    return f"{n}{suffix}"

In [83]:
def read_textbook(subject, chapter, num_pgs=3, include_imgs=False) :
    """Function to read out a textbook to the student"""
    print("In read_textbook")
    text_nodes = get_nodes_by_meta(
        {
            "subject": subject,
            "chapter": chapter,
            "doc_type": "Text"
        }
    )
    
    if include_imgs :
        img_nodes = get_nodes_by_meta(
            {
                "subject": subject,
                "chapter": chapter,
                "doc_type": "Image"
            }
        )
    else :
        img_nodes = []
    
    response = ""
    for node in text_nodes[:num_pgs] :
        pg_no = node.metadata["page_number"]
        response += f"\n\nStarting page {pg_no}..\n\n"
        response += node.text + "\n\n"
        if img_nodes :
            response += "\nThere are {num_imgs} images in page {pg_no}\n\n"

            img_count = 0
            while img_nodes[0].metadata["page_number"] == pg_no :
                img_count += 1
                response += f"The {number_to_ordinal(img_count)} image is:\n"
                response += img_nodes[0].text + "\n\n"
                img_nodes.pop(0)

            response = response.format(
                num_imgs = img_count,
                pg_no = pg_no
            )
        
        response += f"This is the end of page {pg_no}\n"
    
        
    return str(response)

In [86]:
# Readig out pages of the textbook
print(read_textbook("Science", "Chapter10", num_pgs=3, include_imgs=True))

In read_textbook


Starting page 1..

10 Electric Current and its Effects

You might have tried the game 'How steady is your hand?' suggested in Chapter 9 of Class VI. If not, you may try it out now. Some common electric components can be represented by symbols. In Table 10.1, some electric components and their symbols are shown. You may come across different symbols for these components in different books. However, in this book, we shall be using the symbols shown here.

Paheli and Boojho had also set up the game by connecting an electric circuit as suggested in Class VI. They had lots of fun trying it out with their families and friends. They enjoyed it so much that they decided to suggest it to a cousin of theirs who stayed in a different town. So, Paheli made a neat drawing showing how the various electric components were to be connected (Fig. 10.1).

Look at the symbols carefully. In the symbol for the electric cell, notice that there is a longer line and a shorter but thicker par

In [108]:
wfw_tool = FunctionTool.from_defaults(
    fn=read_textbook,
    description=(
        "Used to read out textbooks or certain parts of textbooks"
        "Do not summarize or simplify the output if this tool is used and return it WORD FOR WORD"
        "If the user query mentions keywords like 'read-out', 'read for me' or something similar, this tool is used"
        "The input to the tool should be in a JSON format representing the kwargs (e.g. {{'subject': 'English', 'chapter': 'Chapter1', 'num_pgs': 3, 'include_imgs': False}})"
    )
)

#### c) For Quiz mode

In [88]:
quiz_sys_prompt = """
## INSTRUCTIONS
You are a quiz master whose sole responsibility is to create a set of **random multiple-choice questions (MCQs)** based on the context provided in the section titled "CONTEXT." 
The purpose of these questions is to test the user's knowledge of the subject comprehensively.

Try to ask questions on different subjects and include as much variety as possible. Questions must come from
different themes and topics in the context as far as possible.

You are to generate exactly {num_qns} questions based only on the provided context. 
The generated questions and options must strictly follow the format specified in the section titled "OUTPUT FORMAT."


### RULES
1. Do NOT include any text or explanation other than what is explicitly specified in "OUTPUT FORMAT".
2. Ensure all generated questions are strictly related to the "CONTEXT" and avoid introducing external information.
3. The questions should vary in type (e.g., conceptual, factual, or application-based) to test a range of understanding.
4. Each question must have exactly four options. Ensure that:
   - Only **one option is correct**.
   - The other three options are plausible yet incorrect.
5. The difficulty level of the questions should be balanced across the set (easy, moderate, and challenging).
6. The numbering of questions must range from 1 to {num_qns}.


### OUTPUT FORMAT
[
    {{
        "question_no": <Question number from 1 to {num_qns}>,
        "question": <A relevant question based on the provided context>,
        "option1": <The first option>,
        "option2": <The second option>,
        "option3": <The third option>,
        "option4": <The fourth option>,
        "correct_option": <The correct option from "option1", "option2", "option3", or "option4">
    }},
]

### CONTEXT
{context_str}
"""

In [98]:
def generate_quiz_qns(subject, chapter, num_qns=10) :
    """Function to create a set of random MCQ quiz/test questions"""
    print("In generate_quiz_qns")
    text_nodes = get_nodes_by_meta(
        {
            "subject": subject,
            "chapter": chapter,
            "doc_type": "Text"
        }
    )
    
    context_str = "\n\n".join([node.text for node in text_nodes])
    
    prompt = quiz_sys_prompt.format(
        context_str=context_str,
        num_qns=num_qns
    )
    start_time = time.time()
    llm_response = Settings.llm.complete(prompt).text
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"The LLm call took {elapsed_time:.4f} seconds to run.")
    
    # Extracting the questions from the response
    pattern = r'\{\s*"question_no".*?\}'
    matches = re.findall(pattern, llm_response, re.DOTALL)
#     print(matches)
    questions = [json.loads(match) for match in matches]
    
    return questions, llm_response, matches

In [99]:
start_time = time.time()
questions, llm_response, matches = generate_quiz_qns("Science", "Chapter3")
display(questions)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The code took {elapsed_time:.4f} seconds to run.")

In generate_quiz_qns
The LLm call took 3.9091 seconds to run.


[{'question_no': 1,
  'question': 'What is the purpose of a clinical thermometer?',
  'option1': 'To measure the temperature of any object',
  'option2': 'To measure the temperature of the human body only',
  'option3': 'To measure the temperature of hot milk',
  'option4': 'To measure the temperature of the surroundings',
  'correct_option': 'option2'},
 {'question_no': 2,
  'question': 'Why should we not use a clinical thermometer to measure the temperature of hot milk?',
  'option1': 'Because it is not designed for that purpose',
  'option2': 'Because it is not accurate',
  'option3': 'Because it is too expensive',
  'option4': 'Because it is too big',
  'correct_option': 'option1'},
 {'question_no': 3,
  'question': 'What is the range of a clinical thermometer?',
  'option1': 'From -10°C to 110°C',
  'option2': 'From 35°C to 42°C',
  'option3': 'From 0°C to 100°C',
  'option4': 'From -20°C to 50°C',
  'correct_option': 'option2'},
 {'question_no': 4,
  'question': 'What is the proc

The code took 4.2223 seconds to run.


#### d) Other "hooks" (tools to do other tasks lie sending mail or telegram message)

In [100]:
def send_mail(to, body) :
    """Function to send mail"""
    print(f"Sending Mail to {to} with the body:\n {body}")
    return "Your mail has been successfully sent"

send_mail_tool = FunctionTool.from_defaults(
    fn=send_mail,
    description= (
        "Used to send emails."
        "The input to the tool should be in a JSON format representing the kwargs (e.g. {{'to': 'Sarat', 'body': 'Hey, how are you?'}})"
    )
)

In [101]:
def send_telegram(to, body) :
    """Function to send a telegram message"""
    print(f"Sending telegram message to {to} with the body:\n {body}")
    return "Your telegram message has been successfully sent"

send_telegram_tool = FunctionTool.from_defaults(
    fn=send_telegram,
    description= (
        "Used to send messages to contacts on telegram."
        "The input to the tool should be in a JSON format representing the kwargs (e.g. {{'to': 'Sarat', 'body': 'Hey, how are you?'}})"
    )
)

In [102]:
def send_whatsapp(to, body) :
    """Function to send a whatsapp message"""
    print(f"Sending whatsapp message to {to} with the body:\n {body}")
    return "Your whatsapp message has been successfully sent"

send_whatsapp_tool = FunctionTool.from_defaults(
    fn=send_whatsapp,
    description= (
        "Used to send messages to contacts on whatsapp."
        "The input to the tool should be in a JSON format representing the kwargs (e.g. {{'to': 'Sarat', 'body': 'Hey, how are you?'}})"
    )
)

### Setting up the ReAct agent

In [114]:
react_tools = [
    send_whatsapp_tool,
    rag_tool,
    send_mail_tool,
    send_telegram_tool,
    wfw_tool
]

In [115]:
agent = ReActAgent.from_tools(
    tools=react_tools,
    llm=Settings.llm,
    memory=False,
    verbose=True
)

In [116]:
react_system_header_str = """\

You are designed to help with a variety of tasks, from answering questions \
    to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here (Include all the complete observations))]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

If you get an error, you MUST respond using this format:

```
Thought: I got an error/issue
Answer: [Explain the error in detail here]
```

## Additional Rules
- The answer use the informaton in the most recent observation to curate a detailed(if necessary) and clear response
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.
- You MUST obey the rules stipulated in the descriptions of each tool.
- If the tool rag_agent was used, the complete information in its observation must be returned to the user.
- Do NOT summarize or simplify the output of any tools. Return the output of the tool WORD FOR WORD.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
react_system_prompt = PromptTemplate(react_system_header_str)

In [117]:
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
agent.reset()

In [118]:
response = agent.chat("Read out chapter 3 of English?")
print(response)

> Running step e5c985ad-3cf4-42ab-badb-8d2a055cd584. Step input: Read out chapter 3 of English?
Thought: I need to use a tool to help me answer the question.
Action: read_textbook
Action Input: {'subject': 'English', 'chapter': 'Chapter3', 'num_pgs': 3, 'include_imgs': False}
In read_textbook
Observation: 

Starting page 1..

Before you read, here are some pictorial glimpses of the history of our country from 1757 to 1857. These pictures and 'speech bubbles' will help clarify your understanding of the conditions that led to the event known as the First War of Independence in 1857.

At a function in Delhi:

1. The Martyrs: Oh, my countrymen! Let your eyes fill with tears as you recall the sacrifices of India's martyrs.

This is the end of page 1


Starting page 2..

2. The Company's conquests (1757-1849)

With its superior weapons, the East India Company was extending its power in 18th century India. The British Indian princes were short-sighted. "Call the English merchants. They will h